In [2]:
import cv2 as cv
import numpy as np
import sys
import torch
import torchvision
import torchvision.transforms as T
from cv2 import IMREAD_GRAYSCALE, IMREAD_COLOR
from deep_learning_project.load_data import get_transform
from deep_learning_project.net import LinearRegressionNetwork, SecondNeuralNetwork
from torch import nn
from torchvision.transforms import InterpolationMode
import math
import matplotlib.pyplot as plt
import os

c:\python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
model = SecondNeuralNetwork()

model.load_state_dict(torch.load('models/20221026_1659_SecondNeuralNetwork/best_test_weights.pt'))
model.eval()

classes = ['noface', 'face']

In [4]:
def predict(model, img):
    obj = None
    with torch.no_grad():
        # exploit the model
        logits = model(img)
        pred_probab = nn.Softmax(dim=1)(logits)
        y_pred = pred_probab.argmax(1).item() # indice(s) of the maximum value in the tensor
        obj = (y_pred, pred_probab)
    return obj

In [12]:
def load_background_images():
    TEXTURE_FOLDER = os.path.abspath('./deep_learning_project/textures/')

    onlyfiles = [f for f in os.listdir(TEXTURE_FOLDER) if os.path.isfile(os.path.join(TEXTURE_FOLDER, f))]

    img_datas = []

    for filename in onlyfiles:
        img_src = cv.samples.findFile(os.path.join(TEXTURE_FOLDER, filename))
        img_datas.append(cv.imread(img_src, IMREAD_COLOR))
    
    return img_datas

32


In [36]:
def gather_false_positive(background_images, rescale=0.8, threshold=0.8, stride=1):
    images = [] # this array will contain false positives images

    for image in background_images:
        transform = get_transform()
        transformed_image = transform(T.ToPILImage()(image))

        while (True):
            for y in range(0, transformed_image.size()[1] - 36, stride):
                for x in range(0, transformed_image.size()[2] - 36, stride):

                    # crop and preparing the cropped image
                    new_img = transformed_image[:, y:y+36, x:x+36]
                    torch_new_img = new_img.reshape((1, 1, 36, 36))

                    (y_pred, pred_probab) = predict(model, torch_new_img)
                    
                    # 0 = noface, 1 = face
                    if(y_pred == 1 and (threshold == None or pred_probab.squeeze()[1] >= threshold)):
                        images.append(new_img.reshape(36, 36))

            new_height = math.ceil(transformed_image.size()[1] * rescale)
            new_width = math.ceil(transformed_image.size()[2] * rescale)

            # stop the loop if the image is smaller than the retina
            if new_height < 36 or new_width < 36:
                break

            transformed_image = T.Resize((new_height, new_width), interpolation=InterpolationMode.BILINEAR)(transformed_image)
    
    images = torch.stack(images)
    images = images.reshape(40, 1, 36, 36).permute((0, 2, 3, 1)).numpy()
    return images

In [45]:
TEXTURE_FP_FOLDER = os.path.abspath('./deep_learning_project/texturesfp/0/')
def save_images(dir_path, images):

    os.makedirs(dir_path, exist_ok=True)

    for i, img in enumerate(images):
        filename = str(i) + ".pgm"
        cv.imwrite(os.path.join(dir_path, filename), img)